In [1]:
%load_ext autoreload
%autoreload 2

from openeo_udf.api.udf_data import UdfData, RasterCollectionTile, SpatialExtent
import numpy as np


cube = [
    RasterCollectionTile(
        id="Band1",
        extent=SpatialExtent(bottom=50, top=55, left=3, right=4),
        data=np.zeros((3, 30, 40))
    ),
    RasterCollectionTile(
        id="Band2",
        extent=SpatialExtent(bottom=50, top=55, left=3, right=4),
        data=np.ones((3, 30, 40))
    ),
]

cube


In [2]:
data = UdfData({"EPSG":900913}, cube)

data

In [3]:
udf = """
print("Hello UDF!")
print(type(data))
"""

In [4]:
exec(udf, {"data": data})

Hello UDF!
<class 'openeo_udf.api.udf_data.UdfData'>


In [5]:
from convert import udf_to_eopatch
patch = udf_to_eopatch(data)
patch

EOPatch(
  data: {
    Band1: numpy.ndarray(shape=(3, 30, 40, 1), dtype=float64)
    Band2: numpy.ndarray(shape=(3, 30, 40, 1), dtype=float64)
  }
  mask: {}
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {}
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {}
  bbox: BBox(((3.0, 50.0), (4.0, 55.0)), crs=EPSG:4326)
  timestamp: []
)

In [6]:
from convert import eopatch_to_udf
tiles = eopatch_to_udf(patch)
tiles

In [7]:
from eolearn.core import MapFeatureTask, FeatureType

multiply = MapFeatureTask({FeatureType.DATA: ['Band1', 'Band2']},
                          {FeatureType.DATA: ['m1', 'm2']},
                           lambda f: f*200)

patch = multiply(patch)
#patch[(FeatureType.DATA, 'm2')]

In [8]:
names = [name for name in patch.data]
names

['Band1', 'Band2', 'm1', 'm2']